<a href="https://colab.research.google.com/github/ahatasham5/Doctor-AI/blob/main/lamma3_withfinetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Dataset = ruslanmv/ai-medical-chatbot

In [1]:
# Install necessary libraries
!pip install transformers bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 4.8 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [3]:
# Define the model name
model_name = "ruslanmv/Medical-Llama3-8B"

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define the 4-bit quantization configuration
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16)

# Load the model with the appropriate device map
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=bnb_config,
                                             trust_remote_code=True,
                                             use_cache=False,
                                             device_map="auto")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [11]:
def askme(question):
    sys_message = '''
    You are an AI Medical Assistant trained on a vast dataset of health information. Please be thorough and
    provide an informative answer. If you don't know the answer to a specific medical inquiry, advise seeking professional help.
    '''

    # Create the full prompt with clear separation between system, user, and assistant parts
    prompt = f"{sys_message}\n\nUser: {question}\n\nAssistant:"

    # Tokenize the input manually
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Ensure you use the correct device

    # Generate output from the model
    outputs = model.generate(**inputs, max_new_tokens=200, use_cache=True)

    # Decode the response and return it
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the assistant's response (if model includes prompt in the output, strip the rest)
    if "Assistant:" in response_text:
        answer = response_text.split("Assistant:")[-1].strip()
    else:
        answer = response_text.strip()

    return answer

# Example usage
question = '''I'm a 35-year-old male and for the past few months, I've been experiencing fatigue,
increased sensitivity to cold, and dry, itchy skin.
Could these symptoms be related to hypothyroidism?
If so, what steps should I take to get a proper diagnosis and discuss treatment options?'''

# Call the function
print(askme(question))


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Based on the information provided, it appears that you may be experiencing symptoms of hypothyroidism. 
Hypothyroidism is a condition where the thyroid gland does not produce enough thyroid hormone, which can lead to a variety of symptoms such as fatigue, weight gain, dry skin, and hair loss. 
It's important to consult with a healthcare professional to determine the underlying cause of your symptoms and discuss appropriate treatment options. 
In the meantime, you can try to manage your symptoms by eating a balanced diet, exercising regularly, and getting enough sleep. 
If your symptoms persist or worsen, it's best to seek medical attention as soon as possible. 
Remember, self-diagnosis is not recommended and seeking professional help is always the best course of action. 
I hope this information is helpful. Please let me know if you have any further questions or concerns.
